In [1]:
! aws s3 cp segments_ns  s3://monlam.ai.stt/wav16k/ --recursive

upload: segments_ns/STT_NS_M0263_0003_13272_to_15972.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0003_13272_to_15972.wav
upload: segments_ns/STT_NS_M0263_0004_18435_to_20646.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0004_18435_to_20646.wav
upload: segments_ns/STT_NS_M0263_0005_22874_to_24932.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0005_22874_to_24932.wav
upload: segments_ns/STT_NS_M0263_0009_41048_to_45908.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0009_41048_to_45908.wav
upload: segments_ns/STT_NS_M0263_0006_25624_to_29033.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0006_25624_to_29033.wav
upload: segments_ns/STT_NS_M0263_0008_36897_to_40272.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0008_36897_to_40272.wav
upload: segments_ns/STT_NS_M0263_0011_50582_to_53417.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_NS_M0263_0011_50582_to_53417.wav
upload: segments_ns/STT_NS_M0263_0001_8_to_8547.wav to s3://monlam.ai.stt/st

In [2]:
! cp -r segments_ns/* /media/monlamai/SSD/data/new_segments

In [3]:
import sys
sys.path.append('../util')

from util import get_max_db_id
from util import get_time_span

torchvision is not available - cannot save figures


In [4]:
STT_NS_GA = 5
STT_NS_GB = 8
STT_NS_GG = 14
STT_NS_GH = 15

group_id = STT_NS_GB
target_directory = f"segments_ns"
last_db_id = get_max_db_id()

The maximum ID in the 'Task' table is: 453645


In [5]:
from transformers import pipeline
from tqdm.auto import tqdm

from pathlib import Path

target_path = Path(target_directory)

generator = pipeline(model="spsither/wav2vec2_run9.15")
rows = []

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/wav16k/{file.name}", inf , get_time_span(str(file.name))])

  0%|          | 0/4384 [00:00<?, ?it/s]

In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [7]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

{'inference_transcript': {0: 'ཨེ་ནས་རྗེས་ལ་ཨེ་ནས་ཕར་ཨེ་འདྲའི་ཚོང་ཨེ་འདྲའི་ནང་འཛུལ།',
  1: 'ད་མིང་ག་རེ་ད་ད་ཨོ་དེ་མིང་དྲན་པ་བྱསད་ཤག་ཡོད་བ་གཞེས་བཞགས་ཡོད་བ། ད་ཨོ་དེ་ལ་འགྲོ་རེད་ད་མི་འདུག་བ།',
  2: 'ཐ་མ་ནང་མ་ད་་ས་མ་ད་ལ་ལམ་ཤི་ཉིས་ལ་ལ་ག་ཚད་ཤད་སོང་།',
  3: 'ཨེ་འདྲའི་བུ་མོ་སྙིང་ངོས་ཆེན་མོ་ཤ་ཚ་བོ་ཅིག',
  4: '་བྱས། ཨེ་ནས་ནམ་རྒྱུན་འདྲ་བོ་ཆ་བཞག་ན་ད་ང་ཚོ་སྐད་ད་ཕྱེད་ཀ་སེ་འདྲ་བོ་ལ་ཨེ་ནས་ད།',
  5: 'ད་ང་ཚོ་བུ་གུ་ཆུང་ཆུང་ཡིན་དུས་ཡོད་བ་ ག་རེ་ལབ་ན་འཛིན་གྲྭ་ཨེ་ནས།',
  6: 'ད་ང་ཚོ་ཨེ་འདྲའ་ཉིན་གུང་མཚམས་མཚམས་ཚོགས་ཐེངས་སྟོང་པ་འདྲ་བོ་ཡོད་འདྲ་བོ་བྱས་ ཨེ་ནས།',
  7: 'དེ་རིང་ང་ག་དེའི་སྐབ་ལ་བཤད་དགོས་བསམ་སོང་འདྲ་བོ་ལབ་པ་ཡིན་ན་ཨེ་ནས།',
  8: '་་་ཨེ་འདྲ་ཟོ་རེད་ར།',
  9: '་་་་་ད།'},
 'url': {0: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_NS_M0266_0073_419200_to_421680.wav',
  1: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_NS_M0264_0078_461624_to_466129.wav',
  2: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_NS_M0275_0259_1577466_to_1580672.wav',
  3: 'https://d38pmlk0v88drf.cloudf

In [8]:
df['group_id'] = group_id
df['state'] = 'imported'
df['id'] = df.index + last_db_id + 1

In [9]:
df.head()

,file_name,url,inference_transcript,audio_duration,group_id,state,id
0,STT_NS_M0266_0073_419200_to_421680,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཨེ་ནས་རྗེས་ལ་ཨེ་ནས་ཕར་ཨེ་འདྲའི་ཚོང་ཨེ་འདྲའི་ནང...,2.480,8,imported,453646
1,STT_NS_M0264_0078_461624_to_466129,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ད་མིང་ག་རེ་ད་ད་ཨོ་དེ་མིང་དྲན་པ་བྱསད་ཤག་ཡོད་བ་ག...,4.505,8,imported,453647
2,STT_NS_M0275_0259_1577466_to_1580672,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཐ་མ་ནང་མ་ད་་ས་མ་ད་ལ་ལམ་ཤི་ཉིས་ལ་ལ་ག་ཚད་ཤད་སོང་།,3.206,8,imported,453648
3,STT_NS_M0268_0169_1081829_to_1084059,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཨེ་འདྲའི་བུ་མོ་སྙིང་ངོས་ཆེན་མོ་ཤ་ཚ་བོ་ཅིག,2.230,8,imported,453649
4,STT_NS_M0264_0232_1382121_to_1385766,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,་བྱས། ཨེ་ནས་ནམ་རྒྱུན་འདྲ་བོ་ཆ་བཞག་ན་ད་ང་ཚོ་སྐད...,3.645,8,imported,453650


In [ ]:
df['file_name'] = df['file_name'].str.replace(".wav", "")

In [ ]:
df.to_csv(f"segments_ns_{group_id}.csv", index=False)

In [ ]:
df[df['inference_transcript'].apply(lambda x: len(x) < 500)].shape

In [ ]:
df.shape